<a href="https://colab.research.google.com/github/nsstnaka/machine_learning_handson/blob/master/recommendation_matrix_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習ハンズオン（レコメンデーション）

## 1. ハンズオンの概要
[Movielens](https://grouplens.org/datasets/movielens/)のデータセットを使って、行列分解(matrix factorization)によるレコメンデーションを実装します。

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## 2. データ取得

### 2.1. データのダウンロード・展開

In [ ]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -n ml-latest-small.zip

### 2.2. ファイル読込
pandasを使ってファイルを読み込みます。タイムスタンプはUNIX時刻（1970年1月1日からの経過秒数）になっているので、datetime型に変換します。

In [ ]:
rating_path = 'ml-latest-small/ratings.csv'
rating_df = pd.read_csv(rating_path)
rating_df['timestamp'] = pd.to_datetime(rating_df['timestamp'], unit='s')
rating_df.head(10)

全レコード数を確認します。

In [ ]:
len(rating_df)

## 3. 前処理

### 3.1. IDの通し番号への変換

ユーザーIDのユニーク数と最大値を確認します。

In [ ]:
max(rating_df['userId']), len(set(rating_df['userId']))

映画IDのユニーク数と最大値を確認します。

In [ ]:
max(rating_df['movieId']), len(set(rating_df['movieId']))

IDと通し番号の対応付けをdictionary型で作成します。

In [ ]:
user_idx_dic = {user_id: idx for idx, user_id in enumerate(sorted(list(set(rating_df['userId']))))}
movie_idx_dic = {movie_id: idx for idx, movie_id in enumerate(sorted(list(set(rating_df['movieId']))))}

各レコードのユーザーと映画に通し番号を付与します。

In [ ]:
rating_df['userIndex'] = rating_df['userId'].apply(lambda x: user_idx_dic[x])
rating_df['movieIndex'] = rating_df['movieId'].apply(lambda x: movie_idx_dic[x])
rating_df.head(10)

### 3.2. 不要な要素の除去
ユーザーID、映画IDは不要になったので除去（ついでにタイムスタンプも使わないので除去）します。

In [ ]:
rating_df.drop(columns=['userId', 'movieId', 'timestamp'], inplace=True)
rating_df.head(10)

### 3.3. データ分割

データを訓練用とテスト用に分割します。

In [ ]:
train_users, test_users, train_movies, test_movies, train_ratings, test_ratings =\
    train_test_split(rating_df['userIndex'].values, rating_df['movieIndex'].values, rating_df['rating'].values, test_size=0.2)

各データの件数を確認します。

In [ ]:
len(train_users), len(train_movies), len(train_ratings), len(test_users), len(test_movies), len(test_ratings)

## 4. 学習

### 4.1. データセット生成
訓練データを`tf.data.Dataset`に変換します。

In [ ]:
batch_size = 128
train_dataset = tf.data.Dataset.from_tensor_slices(((train_users, train_movies), train_ratings)).shuffle(len(train_users)).batch(batch_size)

### 4.2. 学習モデル構築
Kerasを使って学習モデルを組み立てていきます。

ユーザーベクトル$U$とアイテムベクトル$V$を定義します。ここではベクトルの次元数を50に設定します。

In [ ]:
dim = 50
user_embeddings = tf.keras.layers.Embedding(len(user_idx_dic), dim, name='user_embedding')
movie_embeddings = tf.keras.layers.Embedding(len(movie_idx_dic), dim, name='movie_embedding')

入力（ユーザーおよび映画の通し番号）をKerasの形式で定義します。

In [ ]:
input_user_indices = tf.keras.Input(shape=(1,), dtype='int32', name='user_input')
input_movie_indices = tf.keras.Input(shape=(1,), dtype='int32', name='movie_input')

入力で渡されたユーザーおよび映画の通し番号を、それぞれベクトルに変換します。

In [ ]:
user_emb = user_embeddings(input_user_indices)
user_emb = tf.keras.layers.Flatten(name='user_emb_flatten')(user_emb)
movie_emb = movie_embeddings(input_movie_indices)
movie_emb = tf.keras.layers.Flatten(name='movie_emb_flatten')(movie_emb)

ユーザーのベクトルと映画のベクトルを掛け合わせて、レーティングの予測値を算出します。

In [ ]:
predicted_ratings = tf.keras.layers.dot([user_emb, movie_emb], axes=1, name='dot')

上記の処理をモデル化します。

In [ ]:
model = tf.keras.Model(inputs=[input_user_indices, input_movie_indices], outputs=predicted_ratings)

モデルをコンパイルします。

損失関数には平均二乗誤差(Mean Squared Error)を採用します。
$$
MSE = \frac{1}{m}\sum^m_{k=1}{\left(y_k-\hat{y}_k\right)^2}
$$

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
model.summary()

早期終了を設定します。

決められたエポック数の学習が終わらなくても、学習が収束した（損失が一定以上下がらなくなった）ところで学習を強制的に終了させます。

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=3)

### 4.3. 学習実行
訓練用のデータを最大200エポック学習させます。早期終了により、200エポックより手前で学習が止まる可能性があります。

In [ ]:
model.fit(train_dataset, epochs=200, callbacks=[early_stopping])

## 5. 評価

テストデータにあるユーザーとアイテムを使って評価の予測を行い、実際の評価値との比較を試してみます。

In [ ]:
test_index = 1000
predict_rating = np.squeeze(model.predict((np.array([test_users[test_index]]), np.array([test_movies[test_index]]))))
actual_rating = test_ratings[test_index]
print('predict={:.2f}, actual={:.1f}'.format(predict_rating, actual_rating))